In [1]:
import os
import time
import threading
import ali_speech
import pyaudio
import wave
import jieba
from ali_speech.callbacks import SpeechRecognizerCallback
from ali_speech.constant import ASRFormat
from ali_speech.constant import ASRSampleRate
from get_audio import get_audio
from word_devide import jieba_devide
from find_command import find_cmd

In [16]:
input_filename = "input.wav"               # 麦克风采集的语音输入
input_filepath = "C:/Users/王子维/Desktop/my_test/one_sentence/"              # 输入文件的path
in_path = input_filepath + input_filename


class MyCallback(SpeechRecognizerCallback):
    """
    构造函数的参数没有要求，可根据需要设置添加
    示例中的name参数可作为待识别的音频文件名，用于在多线程中进行区分
    """
    sen=''
    
    def __init__(self, name='input'):
        self._name = name
    def on_started(self, message):
        print('MyCallback.OnRecognitionStarted: %s' % message)
    def on_result_changed(self, message):
        print('MyCallback.OnRecognitionResultChanged: file: %s, task_id: %s, result: %s' % (
            self._name, message['header']['task_id'], message['payload']['result']))
    def on_completed(self, message):
        print('MyCallback.OnRecognitionCompleted: file: %s, task_id:%s, result:%s' % (
            self._name, message['header']['task_id'], message['payload']['result']))
        MyCallback.sen=message['payload']['result']
    def on_task_failed(self, message):
        print('MyCallback.OnRecognitionTaskFailed: %s' % message)
    def on_channel_closed(self):
        print('MyCallback.OnRecognitionChannelClosed')
def process(client, appkey, token):
    audio_name = 'input.wav'
    callback = MyCallback(audio_name)
    recognizer = client.create_recognizer(callback)
    recognizer.set_appkey(appkey)
    recognizer.set_token(token)
    recognizer.set_format(ASRFormat.PCM)
    recognizer.set_sample_rate(ASRSampleRate.SAMPLE_RATE_16K)
    recognizer.set_enable_intermediate_result(False)
    recognizer.set_enable_punctuation_prediction(True)
    recognizer.set_enable_inverse_text_normalization(True)
    try:
        ret = recognizer.start()
        if ret < 0:
            return ret
        print('sending audio...')
        with open(audio_name, 'rb') as f:
            audio = f.read(3200)
            while audio:
                ret = recognizer.send(audio)
                if ret < 0:
                    break
                time.sleep(0.1)
                audio = f.read(3200)
        recognizer.stop()
    except Exception as e:
        print(e)
    finally:
        recognizer.close()
    return MyCallback.sen
def process_multithread(client, appkey, token, number):
    thread_list = []
    for i in range(0, number):
        thread = threading.Thread(target=process, args=(client, appkey, token))
        thread_list.append(thread)
        thread.start()
    for thread in thread_list:
        thread.join()
if __name__ == "__main__":
    client = ali_speech.NlsClient()
    # 设置输出日志信息的级别：DEBUG、INFO、WARNING、ERROR
    client.set_log_level('INFO')
    appkey = '8qMtpb3T6XdGvxAt'
    token = 'ebfcc60d20b446bfae15d33b680f4893'
    sec=1
    i=0
    while (sec==1):
        get_audio(in_path)
        sentence_result=process(client, appkey, token)
        if sentence_result =='':
            sentence_result='错误'
        word_list=jieba_devide(sentence_result)
        cmd=find_cmd(word_list)
        print(cmd)
        time.sleep(21)#具体延迟看stm32的反应速度
        #在此处开始调用串口传递函数传参cmd

********** 开始录音：请在5秒内输入语音
********** 录音结束



2019-06-27 23:21:02,951 alispeech-INFO  _on_open:78 sending start cmd: {"header": {"namespace": "SpeechRecognizer", "appkey": "8qMtpb3T6XdGvxAt", "name": "StartRecognition", "message_id": "2ce7ffb098ef11e997b5d0577be247f7", "task_id": "2ce7ffb198ef11e99a18d0577be247f7"}, "context": {"sdk": {"name": "nls-sdk-python", "version": "2.0.1"}}, "payload": {"format": "pcm", "sample_rate": 16000, "enable_intermediate_result": false, "enable_punctuation_prediction": true, "enable_inverse_text_normalization": true}}


MyCallback.OnRecognitionStarted: {'header': {'namespace': 'SpeechRecognizer', 'name': 'RecognitionStarted', 'status': 20000000, 'message_id': '64a46f14714d4c11a5b70e7bb664671f', 'task_id': '2ce7ffb198ef11e99a18d0577be247f7', 'status_text': 'Gateway:SUCCESS:Success.'}}
sending audio...


2019-06-27 23:21:08,078 alispeech-INFO  stop:198 sending stop cmd: {"header": {"namespace": "SpeechRecognizer", "appkey": "8qMtpb3T6XdGvxAt", "name": "StopRecognition", "message_id": "2ff64c1098ef11e9bb0cd0577be247f7", "task_id": "2ce7ffb198ef11e99a18d0577be247f7"}}


MyCallback.OnRecognitionCompleted: file: input.wav, task_id:2ce7ffb198ef11e99a18d0577be247f7, result:
MyCallback.OnRecognitionChannelClosed0

********** 开始录音：请在5秒内输入语音
********** 录音结束



...

MyCallback.OnRecognitionCompleted: file: input.wav, task_id:42dc6b3598ef11e99236d0577be247f7, result:前进
MyCallback.OnRecognitionChannelClosed1

********** 开始录音：请在5秒内输入语音
********** 录音结束



2019-06-27 23:22:16,532 alispeech-INFO  _on_open:78 sending start cmd: {"header": {"namespace": "SpeechRecognizer", "appkey": "8qMtpb3T6XdGvxAt", "name": "StartRecognition", "message_id": "58c3858a98ef11e9af89d0577be247f7", "task_id": "58c3858b98ef11e99000d0577be247f7"}, "context": {"sdk": {"name": "nls-sdk-python", "version": "2.0.1"}}, "payload": {"format": "pcm", "sample_rate": 16000, "enable_intermediate_result": false, "enable_punctuation_prediction": true, "enable_inverse_text_normalization": true}}


MyCallback.OnRecognitionStarted: {'header': {'namespace': 'SpeechRecognizer', 'name': 'RecognitionStarted', 'status': 20000000, 'message_id': '476ddda17ec641c681c86bc15ea6e846', 'task_id': '58c3858b98ef11e99000d0577be247f7', 'status_text': 'Gateway:SUCCESS:Success.'}}sending audio...



2019-06-27 23:22:21,655 alispeech-INFO  stop:198 sending stop cmd: {"header": {"namespace": "SpeechRecognizer", "appkey": "8qMtpb3T6XdGvxAt", "name": "StopRecognition", "message_id": "5bd1389898ef11e9a134d0577be247f7", "task_id": "58c3858b98ef11e99000d0577be247f7"}}


MyCallback.OnRecognitionCompleted: file: input.wav, task_id:58c3858b98ef11e99000d0577be247f7, result:左转
MyCallback.OnRecognitionChannelClosed3

